In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import numpy as np
from plotnine import *

from sklearn.preprocessing import StandardScaler

from sklearn.cluster import AgglomerativeClustering


from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

from sklearn.metrics import silhouette_score

# make sure you have these to make dendrograms!-------
import scipy.cluster.hierarchy as sch
from matplotlib import pyplot as plt
#-----------------------------------------------------

%matplotlib inline

# Review

Hierarchical clustering (as the name suggests) assumes that the clusters in the data have a hierarchical relationship. For example, in a McDonald's food dataset we could have clusters like: Dessert, Drinks, Sandwiches, Other. Within Sandwiches we could have chicken sandwiches, Burgers, Vegan Sandwiches...within Burgers we could have smaller, lower calorie options, and bigger, more substantial burgers...etc.

Blood cells is another great example of a hierarchical relationship: ![blood hierarchy](https://community.jmp.com/t5/image/serverpage/image-id/16820i93FA5BD273E0A842/image-dimensions/340x314?v=1.0)

Hierarchical *Agglomeretive* Clustering (which we perform here), goes bottom up: every datapoint starts as it's own singleton cluster, and at each step, we merge the two closest clusters together until all data points are in one big cluster. In order to decide which clusters are closest, we need to choose two things:


## Distance Metrics and Linkage Critera

* **distance metric**: this is a measure that helps us define how close together two *data points* are. Euclidean distance is a common distance metric that you may be familiar with, but there is also cosine distance, manhattan distance, hamming distance, and even custom distance functions (like [levenshtein distance](https://en.wikipedia.org/wiki/Levenshtein_distance) between two strings!)

* **linkage criteria**: this is a measure of how close two *clusters* are. Because (most) clusters have more than one point, we need to define what it means for two clusters to be close.
    * **Single Linkage**: the distance between two clusters (A and B) as the minimum distance between any point in A and any point in B

    ![single linkage](https://community.jmp.com/t5/image/serverpage/image-id/16823iF32133201794C0A4/image-dimensions/251x242?v=1.0)
    * **Average Linkage**: the distance between two clusters (A and B) as the average distance between points in A and points in B.

    ![average linkage](https://community.jmp.com/t5/image/serverpage/image-id/16824iDD065DCADD44D5EC/image-dimensions/275x307?v=1.0)
    * **Complete Linkage**: the distance between two clusters (A and B) as the maximum distance between any point in A and any point in B.
    
    ![complete linkage](https://community.jmp.com/t5/image/serverpage/image-id/16825i39A778742E501081/image-dimensions/277x245?v=1.0)
    * **Centroid Method**: the distance between two clusters (A and B) as the distance between their respective mean vectors (centroids).
    
    ![centroid method](https://community.jmp.com/t5/image/serverpage/image-id/16826iFC5E179AEFFF1252/image-dimensions/260x268?v=1.0)
    * **Ward's Method** (default): the distance between two clusters (A and B) as the Sum of Squared Errors when combining the two clusters together.
    
    ![ward's method](https://community.jmp.com/t5/image/serverpage/image-id/16827iA35DD99890489DB2/image-dimensions/253x164?v=1.0)

    * and MORE! You could technically define this anyway you wanted.

  
## Dendrograms

### Diffuse Overlapping Clusters
<img src="https://drive.google.com/uc?export=view&id=1OCGvoe2FtZdIm0NnXbuwDo49E9CDrkIc" width = 600px />

### Highly Separable Clusters
<img src="https://drive.google.com/uc?export=view&id=1xpYaV-Pa1agH7H-OLzRvCcSWU-k78Uzf" width = 600px />




### Let's Try on Our Own

Chat with your table about the following dendrograms. Which do you think have highly separable data? Which do you think might have overlapping clusters? Discuss each dendrogram in detail.

#### 1.
<img src="https://drive.google.com/uc?export=view&id=1ilZW8x11EjSAYub7jFM_kN4DxLMsgvOx" width = 500px />

#### 2.
<img src="https://drive.google.com/uc?export=view&id=1bz5MM_HZe30uLLesgCXZkEhfcgmVHG43" width = 500px />

#### 3.
<img src="https://drive.google.com/uc?export=view&id=1b3VHTE0WqmgtVa8ywtJX6sQkvjVadQ6D" width = 500px />



## Sklearn

Now time to try to build our own using sklearn!

In [2]:
tests_wide = pd.read_csv("https://raw.githubusercontent.com/katherinehansen2/CPSC392Hansen/refs/heads/main/data/testperform.csv")

tests_wide.head()

In [3]:
# set up X/feats
features = ["zero", "one", "two", "three", "four"]
X = tests_wide[features]

# Z-score
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

hac = AgglomerativeClustering(linkage = "ward",
                              metric = "euclidean",
                              distance_threshold=0,
                              n_clusters = None) # come back and change the number of clusters

# fit and get labels
labels = hac.fit_predict(X_scaled)

In [4]:
# from sklearn: https://github.com/scikit-learn/scikit-learn/blob/70cf4a676caa2d2dad2e3f6e4478d64bcb0506f7/examples/cluster/plot_hierarchical_clustering_dendrogram.py
def plot_dendrogram(hac, **kwargs):

    # create the counts of samples under each node
    counts = np.zeros(hac.children_.shape[0])
    n_samples = len(hac.labels_)
    for i, merge in enumerate(hac.children_):
        current_count = 0
        for child_idx in merge:
            if child_idx < n_samples:
                current_count += 1  # leaf node
            else:
                current_count += counts[child_idx - n_samples]
        counts[i] = current_count

    linkage_matrix = np.column_stack(
        [hac.children_, hac.distances_, counts]
    ).astype(float)

    # Plot the corresponding dendrogram
    sch.dendrogram(linkage_matrix, **kwargs)


plot_dendrogram(hac, color_threshold = 5)

Now we'll refit with a specific number of clusters

In [5]:
hac = AgglomerativeClustering(linkage = "ward",
                              metric = "euclidean",
                              n_clusters = 3) # come back and change the number of clusters

# fit and get labels
labels = hac.fit_predict(X_scaled)

# sil scor
print(silhouette_score(X_scaled, labels))

# look at cluster performance
tests_wide["cluster_3"] = labels
gg_list = []
for test in features:
    title = "Test " + test.capitalize() + " Cluster Performance"
    gg_list.append(ggplot(tests_wide, aes(x = "factor(cluster_3)", y = test))
          + geom_boxplot(aes(fill = "factor(cluster_3)")) +
          theme_minimal() +
          scale_fill_discrete(name = "Cluster Assignment") +
          labs(x = "Cluster",
               y = "Score",
               title = title))

In [6]:
gg_list[0]

In [7]:
gg_list[1]

In [8]:
gg_list[2]

In [9]:
gg_list[3]

In [10]:
gg_list[4]

# ICA

## It's Election Day!

If you haven't gotten a chance to vote, and you need some time to do it, please feel free to take this as an opportunity! There is a voting center in AF119 :)



## Practicing HAC

### Makeup example

Let's look at some makeup purchasing data and cluster customers into groups.

Use HAC with cosine distance to cluster customers based on `eyeshadow`, `lipstick`, and `foundation` purchases.

In [ ]:
# load in data
makeup = pd.read_csv("https://raw.githubusercontent.com/katherinehansen2/CPSC392Hansen/refs/heads/main/data/makeup.csv")
makeup.head()

# set up features
features = ["eyeshaddow", "lipstick", "foundation"]

# no need to z-score because it is count data
X = makeup[features]

# create empty model
hac2 = AgglomerativeClustering(# TODO)


# fit model and get labels
labels = hac2.fit_predict(X[features])

# assess
plot_dendrogram(hac2, color_threshold= 0.015)

In [ ]:
# refit with selected number of clusters



In [ ]:
# TODO
# draw spread boxplots for each feature
# this is baseline code, be sure to update for your specific problem

for test in features:
    title = "Test " + test.capitalize() + " Cluster Performance"
    gg_list.append(ggplot(tests_wide, aes(x = "factor(cluster_3)", y = test))
          + geom_boxplot(aes(fill = "factor(cluster_3)")) +
          theme_minimal() +
          scale_fill_discrete(name = "Cluster Assignment") +
          labs(x = "Cluster",
               y = "Score",
               title = title))

In [ ]:
gg_list[0] # do for all items in list

## Building your own distance functions

TODO:
Let's build a few functions that return the distance between two clusters. Each of these functions take in two dataframes, `A` and `B` that contain the datapoints for the two respective clusters (*number of features can vary*).

The functions take in two arguments:

* `A`: an N1 x P dataframe containing the data points in cluster A. (N1 is the number of data points in cluster A; P is the number of features used)
* `B`: an N2 x P dataframe containing the data points in cluster B. (N2 is the number of data points in cluster B; P is the number of features used)


The function should calculate and return the distance between the clusters (assume you're using euclidean distance for all of these) according to their respective linkage criterion (single, average, and complete). Remember a) that you need to calculate the distance between *every* point in `A` and *every* point in `B` and b) [`np.linalg.norm()`](https://numpy.org/doc/stable/reference/generated/numpy.linalg.norm.html).

### TODO: Discuss

1. To calculate the distance between two clusters `A` (N1 x P) and `B` (N2 x P), how many distances (between 2 data points) would you have to calculate?

In [ ]:
### YOUR CODE HERE ###

def single(A,B):
    pass


def average(A,B):
    pass


def complete(A,B):
    pass


### /YOUR CODE HERE ###

In [11]:
# check if your functions are working

df = pd.read_csv("https://raw.githubusercontent.com/katherinehansen2/CPSC392Hansen/refs/heads/main/data/HAC1.csv")

dA = df.loc[df.cluster == "A"] # cluster A
dB = df.loc[df.cluster == "B"] # cluster B
dC = df.loc[df.cluster == "C"] # cluster C

In [12]:
# if complete() is correct, this will print true
completePASS = abs(complete(dA[["x","y"]], dB[["x","y"]]) - 4.718047025872837) <= 0.01
print("Complete:", completePASS)

# if average() is correct, this will print true
averagePASS = abs(average(dA[["x","y"]], dB[["x","y"]]) - 2.734811240314461) <= 0.01
print("Average:", averagePASS)

# if single() is correct, this will print true
singlePASS = abs(single(dA[["x","y"]], dB[["x","y"]]) - 0.7361237342164363) <= 0.01
print("Single:", singlePASS)

TODO: Using the dataset `df` below,

1. plot the clusters using ggplot, color by cluster
2. use your functions `single()`, `average()`, and `complete()` to calculate the distances between each pair of clusters.

3. Look at which clusters are considered "close" and "far" in different methods. Are there differences between which are furthest/closest between methods? What are they?

4. Describe why you think you see these differences.


In [ ]:
# plot


In [ ]:
# calculate distances (this will likely take a few min to run)

### YOUR CODE HERE ###

#---single------
s_AB = ###
s_AC = ###
s_BC = ###

print("AB:", s_AB)
print("AC:", s_AC)
print("BC:", s_BC)
print("\n")
#---average-----
a_AB = ###
a_AC = ###
a_BC = ###

print("AB:", a_AB)
print("AC:", a_AC)
print("BC:", a_BC)
print("\n")
#---complete----
c_AB = ###
c_AC = ###
c_BC = ###

print("AB:", c_AB)
print("AC:", c_AC)
print("BC:", c_BC)
print("\n")

### /YOUR CODE HERE ###


### Exploring Linkage Criteria

Using the predictors listed in `predictors`, perform HAC on the [burger king dataset](https://github.com/katherinehansen2/CPSC392Hansen/blob/main/data/burger-king-items.txt) using sklearn and the following linkage critera:

* single linkage
* average linkage
* complete linkage
* ward's method

TODO:

1. Plot and compare the different clusters/dendrograms that you get. What do you notice is similar/different?

2. Discuss: Think hard: what do the dendrograms tell you about the data?

(NOTE: see [documentation](https://scikit-learn.org/stable/modules/generated/sklearn.cluster.AgglomerativeClustering.html#sklearn.cluster.AgglomerativeClustering) if you need a refresher on how to set linkage criteria in sklearn, and [here](https://docs.scipy.org/doc/scipy/reference/generated/scipy.cluster.hierarchy.linkage.html) for how to set it for the dendrogram)

In [ ]:
burg = pd.read_csv("https://raw.githubusercontent.com/katherinehansen2/CPSC392Hansen/refs/heads/main/data/burger-king-items.txt",
                  sep = "\t")

predictors = ["Calories", "Protein(g)", "Fat(g)", "Sodium(mg)", "Carbs(g)", "Sugar(g)"]

X = burg[predictors]

In [ ]:
# TODO Single Linkage


In [ ]:
# TODO Average Linkage


In [ ]:
# TODO Complete Linkage


In [ ]:
# TODO Ward's Linkage
